<a href="https://colab.research.google.com/github/cpyp-ipdp/pgd_cdmx/blob/main/indicadores/vivienda/indicadores_vivienda_pgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Estimación de indicadores de vivienda para el Plan General de Desarrollo de la Ciudad de México

En este script se descargan los microdatos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) para los años 2016, 2018, 2020 y 2022. Posteriormente, se estiman dos indicadores de vivienda para la Ciudad de México: rezago habitacional y viviendas sin escrituras. Los resultados se presentan en dos cuadros.

In [ ]:
##Estimación de indicadores de vivienda
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr,archive)

Loading required package: pacman



In [ ]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [ ]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_viviendas_csv.zip"

In [ ]:
years<-c("2016", "2018", "2020", "2022")

In [ ]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = "data")

  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/viviendas.csv",
    to = paste0("data/viviendas_", years[i], ".csv")
  )
}

### Función de estimación

In [ ]:
estimacion <- function(path, variable) {
  # Cargar los datos
  datos <- read_csv(path)

  datos <- datos %>%
    mutate(
      across(starts_with("mat") & where(is.character),
             ~ replace_na(parse_number(.x, na = c('', 'NA', '&')), 0)),
      rezago = if_else(
        ((tot_resid / num_cuarto) > 2.5) |
          (mat_pared %in% 1:6) |
          (mat_techos %in% c(1:4, 6, 7, 9)) |
          (mat_pisos == 1) |
          (excusado == 2),
        "En rezago",
        "Fuera de rezago"
      )
    ) %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")

  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estimar rezago habitacional
  rezago <- mydesign %>%
    group_by(rezago) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    mutate(pct = round(pct * 100, 2)) %>%
    filter(rezago == "En rezago") %>%
    # Añadir año
    mutate(year = substr(path, 16, 19))



  # Estimar viviendas sin escrituras
  escrituras <- mydesign %>%
    filter(tenencia %in% c(3,4)) %>%
group_by(escrituras) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    filter(escrituras == 3) %>%
    mutate(pct = round(pct * 100, 2),
           escrituras = case_when(
        escrituras == 3 ~ "Sin escrituras"
))%>%
mutate(year = substr(path, 16, 19))


#Hogares en la vivienda

hogares<-mydesign %>%
summarise(
    hogares=survey_total(tot_hog, vartype = "cv")
    ) %>%
mutate(year = substr(path, 16, 19))





##Hogares hacinamiento

hogares_hac<-mydesign %>%
#Identificar hacinamiento
mutate(hacinamiento=ifelse((tot_resid/num_cuarto)>2.5,1,0))%>%
#filtrar hacinamiento
#filter(hacinamiento==1)%>%
group_by(hacinamiento)%>%
summarise(
    hogares_hac=survey_total(vartype = "cv"),
    pct=survey_prop(vartype = "cv")
    ) %>%
mutate(year = substr(path, 16, 19))%>%
mutate(pct=round(pct*100,2))






#Hog hacinamiento no propietarios
hogares_hac_no_prop<-mydesign%>%
#Identificar hacinamiento
mutate(hacinamiento=ifelse((tot_resid/num_cuarto)>2.5,1,0))%>%
mutate(ten_vivi=case_when(tenencia==3 | tenencia==4 ~ "Propietarios",
            TRUE ~ "No propietarios"))%>%
#filtrar hacinamiento
filter(hacinamiento==1)%>%
group_by(ten_vivi)%>%
#Contar hogares con hacinamiento
summarise(
    hogares_hac=survey_prop(vartype = "cv")
    ) %>%
ungroup()%>%
mutate(year = as.integer(substr(path, 16, 19)))%>%
filter(ten_vivi=="No propietarios")%>%
mutate(hogares_hac=round(hogares_hac*100,2))



  # Devolver el resultado basado en la variable
  if (variable == "rezago") {
    return(rezago)
  } else if (variable == "sin_escrituras") {
    return(escrituras)
  }
else if (variable == "hogares") {
    return(hogares)
  }
  else if (variable == "hogares_hac_no_prop") {
    return(hogares_hac_no_prop)
}
else if (variable == "hogares_hac") {
    return(hogares_hac)

  }

else {
    stop("Variable no reconocida. Use 'rezago' o 'sin_escrituras'.")
  }}


In [ ]:
#Construir cuadros de rezago habitacional

tabla_rezago<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
  estimacion,
  variable = "rezago"
)
print("Porcentaje de viviendas cuyos habitantes viven en rezago habitacional")
tabla_rezago


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

[1] "Porcentaje de viviendas cuyos habitantes viven en rezago habitacional"


rezago,viviendas,viviendas_cv,pct,pct_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
En rezago,272200,0.08809435,9.81,0.08684886,2016
En rezago,218501,0.08906203,8.02,0.08727815,2018
En rezago,198021,0.08360376,7.31,0.08292165,2020
En rezago,175242,0.09933936,5.94,0.09870452,2022


In [ ]:
#Construir cuadros de viviendas sin escrituras
#Viviendas sin escrituras corresponden a viviendas propias y propias que se están pagando y no cuentan con escrituras

tabla_escrituras<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable = "sin_escrituras"
    )
print("Porcentaje de viviviendas propias o propias en proceso de pago que no cuentan con escrituras")
tabla_escrituras

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

[1] "Porcentaje de viviviendas propias o propias en proceso de pago que no cuentan con escrituras"


escrituras,viviendas,viviendas_cv,pct,pct_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Sin escrituras,266547,0.08646642,17.71,0.08298775,2016
Sin escrituras,246283,0.09985847,17.22,0.09461881,2018
Sin escrituras,259056,0.09464920,16.23,0.08765464,2020
Sin escrituras,310191,0.08428046,17.63,0.07835935,2022


In [ ]:
tabla_hogares<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable = "hogares"
    )%>%
mutate(
  year = as.integer(year),
tasa_crec=(hogares/lag(hogares)-1)*100)
tabla_hogares

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

hogares,hogares_cv,year,tasa_crec
<dbl>,<dbl>,<int>,<dbl>
2825100,0.01357273,2016,NA
2778842,0.01510168,2018,-1.637393
2731683,0.01285271,2020,-1.697074
2990030,0.01388237,2022,9.457430


In [ ]:
tabla_hogares_hac<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable="hogares_hac"
    )%>%
mutate(
  year = as.integer(year),
  #asa_crec = (hogares_hac / lag(hogares_hac) - 1) * 100
)%>%
filter(hacinamiento==1)
print("Porcentaje de hogares en hacinamiento")
tabla_hogares_hac

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

[1] "Porcentaje de hogares en hacinamiento"


hacinamiento,hogares_hac,hogares_hac_cv,pct,pct_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,94467,0.1494674,3.40,0.1489454,2016
1,69444,0.1668508,2.55,0.1655767,2018
1,79858,0.1226073,2.95,0.1215774,2020
1,50615,0.1625358,1.72,0.1624582,2022


In [ ]:
tabla_hogares_hac_no_prop<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable="hogares_hac_no_prop"
    )
print("Porcentaje de hogares en hacinamiento que no son propietarios de la vivienda")
tabla_hogares_hac_no_prop

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

[1] "Porcentaje de hogares en hacinamiento que no son propietarios de la vivienda"


ten_vivi,hogares_hac,hogares_hac_cv,year
<chr>,<dbl>,<dbl>,<int>
No propietarios,81.57,0.07178577,2016
No propietarios,83.33,0.06234368,2018
No propietarios,78.57,0.06250299,2020
No propietarios,72.08,0.10327248,2022


### Indicador de migración por causas relacionadas con ubicación o vivienda


Para este indicador se utilizan los datos de migración del Censo. Concretamente, se consideran aquellos que cambiaron de residencia en los últimos cinco años y que se mudaron por causas relacionadas con ubicación o vivienda.

In [ ]:
#Descarga de población de Censo 2020

url_censo<-"https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_eum_csv.zip"

In [ ]:
# Verificar si existe el archivo Personas00.CSV
if (!file.exists("data/Personas00.CSV")) {
  # Descarga en carpeta data
  destfile_censo <- "data/Censo2020_CA_eum_csv.zip"

  # Configuración de tiempo de espera para la descarga
  options(timeout = 900)
  download.file(url_censo, destfile = destfile_censo, quiet = FALSE, mode = "wb")

  # Ruta a WinRAR (asegúrate de que sea la correcta)
  winrar <- shQuote("C:/Program Files/WinRAR/WinRAR")
  destino <- paste0(getwd(), "/data/")

  # Ejecutar extracción
  cmd <- paste(winrar, 'e', shQuote(destfile_censo), '-r', shQuote(destino), sep = ' ')
  system(cmd)
} else {
  message("El archivo Personas00.CSV ya existe en la carpeta data. Proceso omitido.")
}


El archivo Personas00.CSV ya existe en la carpeta data. Proceso omitido.



In [ ]:
pob<-read_csv("data/Personas00.CSV")%>%
janitor::clean_names()%>%
filter(ent_pais_res_5a <= "032") %>%
#Sustraer 2 dígitos de la clave de la entidad
mutate(ent_pais_res_5a = substr(ent_pais_res_5a, 2, 3))%>%
   mutate(ent_pais_res_5a = case_when(
     nchar(as.character(ent_pais_res_5a)) == 1 ~ paste0("0", ent_pais_res_5a),
     TRUE ~ as.character(ent_pais_res_5a)
   )) %>%

  mutate(ent = case_when(
  nchar(as.character(ent)) == 1 ~ paste0("0", ent),TRUE ~ as.character(ent)))%>%
 # No contabilizar la misma entidad
    filter(ent_pais_res_5a=="09")%>%
filter(ent!="09")%>%
  filter(edad>=5)%>%

##Se identifican las causas de migración relacionadas con la vivienda y ubicación
#El catálogo de causas se puede descargar de https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/nueva_estruc/702825198701.pdf

mutate(causa_vivi=
case_when(causa_mig_v %in% c("0104","0201","0503","0901","0902", "0903","0905","0909") ~ "Causas relacionadas con ubicación o vivienda",
TRUE ~ "Otra causa"))

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 15015683 Columns: 91
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (33): ENT, MUN, LOC50K, ID_VIV, ID_PERSONA, ESTRATO, UPM, CLAVIVP, NUMPE...
dbl (57): COBERTURA, FACTOR, SEXO, EDAD, PARENTESCO, NACIONALIDAD, AFRODES, ...
lgl  (1): MED_TRASLADO_ESC3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
#Opción para ajustar si se tiene una sola upm
options(survey.lonely.psu="certainty")
#Definir diseño muestral

dm<-pob%>%
  #Diseño muestral
  as_survey_design(weights=factor, strata=estrato, ids=upm, nest=TRUE)

In [ ]:
print("Personas que salieron de CDMX de 2015 a 2020 por causas relacionadas con la ubicación o vivienda")
dm%>%
group_by(causa_vivi)%>%
    summarise(personas=survey_total(vartype="cv"),
              porcentaje=survey_prop(vartype="cv"))%>%
    mutate(porcentaje=round(porcentaje*100,2))

[1] "Personas que salieron de CDMX de 2015 a 2020 por causas relacionadas con la ubicación o vivienda"


causa_vivi,personas,personas_cv,porcentaje,porcentaje_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Causas relacionadas con ubicación o vivienda,42329,0.05677074,7.48,0.049071527
Otra causa,523922,0.01806808,92.52,0.003964614


#### Indicador de pobreza por calidad y espacios de la vivienda de Evalúa CDMX

In [1]:

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, haven, googledrive)

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’, ‘RcppArmadillo’, ‘survey’



srvyr installed



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
  #Conectar a google drive
drive_auth(email = "claudiodanielpc@gmail.com",cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=82457bbdce82c5ef25bda511fe1c26d8&login_hint=claudiodanielpc%40gmail.com&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVNWZ2kzSjA4amRDZzBTcG1vOEVYeUkxcnVfb2pqOWphaWZEZGZfd3pCOVlMeURiRVdxWmdHZVpaUVYxWFBvUmh5dm5BQSIsInN0YXRlIjoiODI0NTdiYmRjZTgyYzVlZjI1YmRhNTExZmUxYzI2ZDgifQ==


In [4]:
carpeta<-as_id(drive_get("https://drive.google.com/drive/folders/1TBfoik4w4DWhwFqWeZFgkZvrBdDbqzku"))


archivos<-drive_ls(carpeta)
print(archivos)

# A dribble: 3 × 3
  name             id                                drive_resource   
  <chr>            <drv_id>                          <list>           
1 pobreza_2022.dta 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan <named list [42]>
2 pobreza_2020.dta 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx <named list [42]>
3 pobreza_2018.dta 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL <named list [42]>


In [5]:
estimacion_pobreza <- function(file_id, file_name) {
  # Descarga de datos
  local_path <- file.path("data", file_name)
  drive_download(as_id(file_id), path = local_path, overwrite = TRUE)

  # Leer datos
  datos <- read_dta(local_path)

  # Añadir año
  year <- str_extract(file_name, "\\d{4}")


  # Diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estima pobreza
  pobreza <- mydesign %>%
    #Filtrar datos para CDMX
  filter(entidad==9)%>%
    #Filtar grupo de población en pobreza
   group_by(pobreza_ccevj)%>%
    summarise(
      poblacion = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = as.numeric(year))%>%
            filter(pobreza_ccevj == 1)

  return(pobreza)
}

In [6]:
tabla_pobreza <- map_dfr(
  seq_len(nrow(archivos)),
  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x])
)%>%
  arrange(year)

print("Estimación de población en situación de pobreza por calidad y espacios de la vivienda en la CDMX")
tabla_pobreza

File downloaded:

• pobreza_2022.dta <id: 1T_WfYanYMOoWWbhj_CeiqxYRNrpXmDan>

Saved locally as:

• data/pobreza_2022.dta

File downloaded:

• pobreza_2020.dta <id: 1TXJLNA_itWfjsGIdoj1bJQ4erNjc1wIx>

Saved locally as:

• data/pobreza_2020.dta

File downloaded:

• pobreza_2018.dta <id: 1TTRfiRDGWgQwHylvduHx1IRdm5vQ3VeL>

Saved locally as:

• data/pobreza_2018.dta



[1] "Estimación de población en situación de pobreza por calidad y espacios de la vivienda en la CDMX"


pobreza_ccevj,poblacion,poblacion_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4510741,0.04025664,49.05,0.02873110,2018
1,4456576,0.03860316,48.23,0.02895216,2020
1,3968378,0.04693254,42.65,0.03663865,2022
